In [36]:
#@title Install dependencies
!pip -q install meds_etl==0.3.6 meds_transforms==0.0.9 es-aces==0.6.1

In [37]:
#@title Download MIMIC-IV demo
import tempfile
import os
from pathlib import Path
temp_dir = tempfile.TemporaryDirectory()
notebook_dir = os.getcwd()

ROOT_DIR=f"{notebook_dir}/work_dir/mimiciv_demo/"
Path(ROOT_DIR).mkdir(parents=True, exist_ok=True)
!echo {ROOT_DIR}

/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/


In [38]:
!wget -q -r -N -c --no-host-directories --cut-dirs=1 -np -P {ROOT_DIR}/raw_data https://physionet.org/files/mimic-iv-demo/2.2/

In [39]:
# Download pre-meds script, event config (defining how raw data is converted to meds data), and meds-transform config
!mkdir {ROOT_DIR}/meds-transform/
!git clone --depth 1 https://github.com/mmcdermott/MEDS_transforms.git {ROOT_DIR}/tmp/
!mv {ROOT_DIR}/tmp/MIMIC-IV_Example {ROOT_DIR}/MIMIC-IV_Example
!cp {ROOT_DIR}/MIMIC-IV_Example/

Cloning into '/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tmp'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 144 (delta 22), reused 70 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (144/144), 211.41 KiB | 571.00 KiB/s, done.
Resolving deltas: 100% (22/22), done.
usage: cp [-R [-H | -L | -P]] [-fi | -n] [-aclpSsvXx] source_file target_file
       cp [-R [-H | -L | -P]] [-fi | -n] [-aclpSsvXx] source_file ... target_directory


In [40]:
# download MIMIC IV metadata
MIMICIV_RAW_DIR = "https://raw.githubusercontent.com/MIT-LCP/mimic-code/v2.4.0/mimic-iv/concepts/concept_map"
MIMICIV_PRE_MEDS_DIR = f"{ROOT_DIR}/pre_meds/"
!mkdir {MIMICIV_PRE_MEDS_DIR}

OUTPUT_DIR = f"{ROOT_DIR}/raw_data/mimic-iv-demo/2.2"

files = [
    'd_labitems_to_loinc.csv',
    'inputevents_to_rxnorm.csv',
    'lab_itemid_to_loinc.csv',
    'meas_chartevents_main.csv',
    'meas_chartevents_value.csv',
    'numerics-summary.csv',
    'outputevents_to_loinc.csv',
    'proc_datetimeevents.csv',
    'proc_itemid.csv',
    'waveforms-summary.csv'
]

for file in files:
    !wget -O {OUTPUT_DIR}/{file} {MIMICIV_RAW_DIR}/{file}
    !wget -O {MIMICIV_PRE_MEDS_DIR}/{file} {MIMICIV_RAW_DIR}/{file}

--2024-12-14 17:15:14--  https://raw.githubusercontent.com/MIT-LCP/mimic-code/v2.4.0/mimic-iv/concepts/concept_map/d_labitems_to_loinc.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 361048 (353K) [text/plain]
Saving to: ‘/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//raw_data/mimic-iv-demo/2.2/d_labitems_to_loinc.csv’

/Users/sim/Document 100%[===================>] 352.59K  1.04MB/s    in 0.3s    

2024-12-14 17:15:14 (1.04 MB/s) - ‘/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//raw_data/mimic-iv-demo/2.2/d_labitems_to_loinc.csv’ saved [361048/361048]

--2024-12-14 17:15:15--  https://raw.githubusercontent.com/MIT-LCP/mimic-code/v2.4.0/mimic-iv/concepts/concept_map/d_labitems_to_loinc.csv
Resolving raw.githubuserc

In [41]:
!cp configs/extract_MIMIC.yaml {ROOT_DIR}/MIMIC-IV_Example/configs/.

In [42]:
# Convert to MEDS
TUTORIAL_DIR = f"{ROOT_DIR}/MIMIC-IV_Example"
MIMICIV_RAW_DIR = f"{ROOT_DIR}/raw_data/mimic-iv-demo/2.2"
MIMICIV_PRE_MEDS_DIR = f"{ROOT_DIR}/pre_meds/"
MIMICIV_MEDS_DIR = f"{ROOT_DIR}/meds/"

EVENT_CONVERSION_CONFIG_FP=f"{ROOT_DIR}/MIMIC-IV_Example/configs/event_config.yaml"
PIPELINE_CONFIG_PATH=f"{ROOT_DIR}/MIMIC-IV_Example/configs/pipeline_config.yaml"
!echo {TUTORIAL_DIR}
!cd {TUTORIAL_DIR} && bash {TUTORIAL_DIR}/run.sh {MIMICIV_RAW_DIR} {MIMICIV_PRE_MEDS_DIR} {MIMICIV_MEDS_DIR} do_unzip=true

/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//MIMIC-IV_Example
Unsetting SLURM_CPU_BIND in case you're running this on a slurm interactive node with slurm parallelism
Setting DO_UNZIP=true
Unzipping csv.gz files matching /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//raw_data/mimic-iv-demo/2.2/*/*.csv.gz.
Running pre-MEDS conversion.
2024-12-14 17:15:27.980 | DEBUG    | MEDS_transforms.extract.utils:get_supported_fp:126 - Found file: /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/raw_data/mimic-iv-demo/2.2/hosp/poe_detail.csv
2024-12-14 17:15:27.980 | INFO     | __main__:main:263 - No function needed for hosp/poe_detail: Symlinking /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/raw_data/mimic-iv-demo/2.2/hosp/poe_detail.csv to /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/pre_meds/hosp/poe_detail.csv
2024-12-14 17:15:27.981 | DEBUG    | MEDS_transforms.extract.utils:get_supported_fp:126 -

In [43]:
#@title Examine MEDS data

import polars as pl
data = pl.read_parquet(f'{ROOT_DIR}/meds/data/**/*.parquet')

data[['subject_id', 'time', 'code', 'numeric_value']]

subject_id,time,code,numeric_value
i64,datetime[μs],str,f32
10001725,null,"""GENDER//F""",null
10001725,2064-01-01 00:00:00,"""MEDS_BIRTH""",null
10001725,2109-10-15 00:00:00,"""Weight (Lbs)""",null
10001725,2109-10-15 00:00:00,"""Blood Pressure""",null
10001725,2110-01-04 00:00:00,"""Weight (Lbs)""",null
…,…,…,…
10039708,2145-01-04 11:47:00,"""LAB//50934//UNK""",33.0
10039708,2145-01-04 11:47:00,"""LAB//50868//mEq/L""",15.0
10039708,2145-01-04 12:24:00,"""LAB//51274//sec""",13.4


In [44]:
#@title A Simple Polars Analysis

icd10_events = data.filter(pl.col('code').str.starts_with('DIAGNOSIS//ICD//10//'))

icd10_events.group_by('code').count().sort('count', descending=True)

/var/folders/fm/gwbmjrfx6gxg6x7xxcxbnrth0000gn/T/ipykernel_61345/1492712476.py:5: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  icd10_events.group_by('code').count().sort('count', descending=True)


code,count
str,u32
"""DIAGNOSIS//ICD//10//E785""",57
"""DIAGNOSIS//ICD//10//E039""",47
"""DIAGNOSIS//ICD//10//Z794""",37
"""DIAGNOSIS//ICD//10//Z87891""",35
"""DIAGNOSIS//ICD//10//I2510""",33
…,…
"""DIAGNOSIS//ICD//10//Z792""",1
"""DIAGNOSIS//ICD//10//E041""",1
"""DIAGNOSIS//ICD//10//Z4502""",1


In [45]:
df = pl.read_parquet(f"{ROOT_DIR}/meds/metadata/codes.parquet")
df

code,description,parent_codes,itemid,possibly_cpt_code,valueuom
str,str,list[str],list[str],list[str],list[str]
"""DIAGNOSIS//ICD//10//E873""","""Alkalosis""","[""ICD10CM/E87.3""]",[null],[null],[null]
"""DIAGNOSIS//ICD//10//Z85810""","""Personal history of malignant …","[""ICD10CM/Z85.810""]",[null],[null],[null]
"""LAB//50883//mg/dL""","""Bilirubin.direct [Mass/volume]…","[""LOINC/1968-7""]","[""50883""]",[null],"[""mg/dL""]"
"""LAB//51000//mg/dL""","""Triglyceride [Mass/volume] in …","[""LOINC/2571-8""]","[""51000""]",[null],"[""mg/dL""]"
"""INFUSION_END//227531""","""mannitol Injection""","[""RxNorm/1791383""]","[""227531""]",[null],[null]
…,…,…,…,…,…
"""LAB//51501//#/hpf""","""Transitional cells [#/area] in…","[""LOINC/30089-7""]","[""51501""]",[null],"[""#/hpf""]"
"""DIAGNOSIS//ICD//9//V600""","""Lack of housing""","[""ICD9CM/V60.0""]",[null],[null],[null]
"""PROCEDURE//START//225400""","""Bronchoscopy""","[""SNOMED/10847001""]","[""225400""]",[null],[null]


In [46]:
#@title Download E-ICU demo
import tempfile
import os
from pathlib import Path
notebook_dir = os.getcwd()

ROOT_DIR=f"{notebook_dir}/work_dir/eicu_demo/"
Path(ROOT_DIR).mkdir(parents=True, exist_ok=True)

!echo {ROOT_DIR}

/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/eicu_demo/


In [47]:
# Download pre-meds script, event config (defining how raw data is converted to meds data), and meds-transform config
!mkdir {ROOT_DIR}/meds-transform/
!git clone --depth 1 https://github.com/mmcdermott/MEDS_transforms.git {ROOT_DIR}/tmp/
!mv {ROOT_DIR}/tmp/eICU_Example {ROOT_DIR}/eICU_Example
# Override configs, we remove the column apneaparms and the whole infusionDrug table as they are not in the demo
!cp configs/extract_eICU.yaml {ROOT_DIR}/eICU_Example/configs/.
!cp configs/table_preprocessors.yaml {ROOT_DIR}/eICU_Example/configs/.
!cp configs/event_configs.yaml {ROOT_DIR}/eICU_Example/configs/.

Cloning into '/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/eicu_demo//tmp'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 144 (delta 22), reused 70 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (144/144), 211.41 KiB | 478.00 KiB/s, done.
Resolving deltas: 100% (22/22), done.


In [48]:
!wget -q -r -N -c --no-host-directories --cut-dirs=1 -np -P {ROOT_DIR}/raw_data https://physionet.org/files/eicu-crd-demo/2.0.1/

In [ ]:
# Convert to MEDS
TUTORIAL_DIR = f"{ROOT_DIR}/eICU_Example"
MIMICIV_RAW_DIR = f"{ROOT_DIR}/raw_data/eicu-crd-demo/2.0.1"
MIMICIV_PRE_MEDS_DIR = f"{ROOT_DIR}/pre_meds/"
MIMICIV_MEDS_DIR = f"{ROOT_DIR}/meds/"

EVENT_CONVERSION_CONFIG_FP=f"{ROOT_DIR}/eICU_Example/configs/event_config.yaml"
PIPELINE_CONFIG_PATH=f"{ROOT_DIR}/eICU_Example/configs/pipeline_config.yaml"
!echo {TUTORIAL_DIR}
!cd {TUTORIAL_DIR} && bash run.sh {MIMICIV_RAW_DIR} {MIMICIV_PRE_MEDS_DIR} {MIMICIV_MEDS_DIR} do_unzip=true

/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/eicu_demo//eICU_Example
Note that eICU has a lot more observations per subject than does MIMIC-IV, so to keep to a reasonable 
memory burden (e.g., < 150GB per worker), you will want a smaller shard size, as well as to turn off 
the final unique check (which should not be necessary given the structure of eICU and is expensive) 
in the merge stage. You can do this by setting the following parameters at the end of the mandatory 
args when running this script:
  * stage_configs.split_and_shard_subjects.n_subjects_per_shard=10000
  * stage_configs.merge_to_MEDS_cohort.unique_by=null
Additionally, consider reducing N_PARALLEL_WORKERS if > 1
Skipping unzipping.
Running pre-MEDS conversion.
2024-12-14 17:20:07.072 | INFO     | __main__:main:278 - Loading table preprocessors from /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/eicu_demo/eICU_Example/configs/table_preprocessors.yaml...
2024-12-14 17:20:07.090 | INFO     | __main__:main:2

In [50]:
#@title Examine MEDS data

import polars as pl
data = pl.read_parquet(f'{ROOT_DIR}/meds/data/**/*.parquet')

data[['subject_id', 'time', 'code', 'numeric_value']]

subject_id,time,code,numeric_value
i64,datetime[μs],str,f32
131736,null,"""GENDER//Female""",null
131736,null,"""ETHNICITY//Caucasian""",null
131736,1982-07-02 02:07:00,"""MEDS_BIRTH""",null
131736,2015-12-30 22:52:00,"""LAB//mg/dL//mg/dL//glucose""",99.0
131736,2015-12-30 22:52:00,"""LAB//mg/dL//mg/dL//calcium""",8.8
…,…,…,…
2736241,2015-12-31 13:00:00,"""NURSE_CHARTING//ENTERED//NOT Y…",null
2736241,2015-12-31 13:00:00,"""RESP_CARE//STATUS//NOT YET DON…",null
2736241,2015-12-31 19:29:00,"""HOSPITAL_DISCHARGE//Alive//Hom…",null
